# Integrated Gradients Benchmark

In [1]:
import sys
sys.path.append("../")

import fastISM
from fastISM.models.basset import basset_model

from fastISM.models.factorized_basset import factorized_basset_model
from fastISM.models.bpnet import bpnet_model
import tensorflow as tf
import numpy as np
from importlib import reload
import time

In [2]:
reload(fastISM.flatten_model)
reload(fastISM.models)
reload(fastISM.ism_base)
reload(fastISM.change_range)
reload(fastISM.fast_ism_utils)
reload(fastISM)

<module 'fastISM' from '../fastISM/__init__.py'>

In [3]:
tf.__version__

'2.3.0'

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
device = 'GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/device:CPU:0'
device

'GPU:0'

In [6]:
import alibi
from alibi.explainers import IntegratedGradients

In [7]:
alibi.__version__

'0.5.3'

## Benchmark

In [8]:
def time_ig(model, batch_sizes, seqlen, num_examples=500, n_steps=50, targets = [None]):
    x = np.random.random((num_examples,seqlen,4))
    times = []
    per_100 = []
    for b in batch_sizes:
        ig  = IntegratedGradients(model,
                              layer=None,
                              method="gausslegendre",
                              n_steps=n_steps,
                              internal_batch_size=b)
        # dry run
        ig.explain(x[:10], baselines=None,
                   target=targets[0])
        
        t = time.time()
        for tgt in targets:
            ig.explain(x, baselines=None,
                       target=tgt)
        times.append(time.time()-t)
        per_100.append((times[-1]/num_examples)*100)
        print("BATCH: {}\tTIME: {:.2f}\tPER 100: {:.2f}".format(b, times[-1], (times[-1]/num_examples)*100))
    
    print("BEST PER 100: {:.2f}".format(min(per_100)))

### Basset (1000)

In [13]:
model = basset_model(seqlen=1000, num_outputs=1)

In [20]:
%%capture --no-stdout 
# hide warning about scalar output

time_ig(model, [100, 200, 500], 1000, num_examples=10, targets=[None]) # targets None since only one scalar output

BATCH: 100	TIME: 0.37	PER 100: 3.73
BATCH: 200	TIME: 0.36	PER 100: 3.61
BATCH: 500	TIME: 0.36	PER 100: 3.57
BEST PER 100: 3.57


In [21]:
%%capture --no-stdout

time_ig(model, [100, 200, 500, 1000], 1000, num_examples=100, targets=[None]) 

BATCH: 100	TIME: 3.77	PER 100: 3.77
BATCH: 200	TIME: 3.64	PER 100: 3.64
BATCH: 500	TIME: 3.58	PER 100: 3.58
BATCH: 1000	TIME: 3.59	PER 100: 3.59
BEST PER 100: 3.58


### Basset (2000)

In [22]:
model = basset_model(seqlen=2000, num_outputs=1)

In [23]:
%%capture --no-stdout

time_ig(model, [100, 200, 500], 2000, num_examples=10, targets=[None]) 

BATCH: 100	TIME: 0.74	PER 100: 7.40
BATCH: 200	TIME: 0.72	PER 100: 7.21
BATCH: 500	TIME: 0.72	PER 100: 7.18
BEST PER 100: 7.18


In [24]:
%%capture --no-stdout

time_ig(model, [100, 200, 500], 2000, num_examples=100, targets=[None]) 

BATCH: 100	TIME: 7.39	PER 100: 7.39
BATCH: 200	TIME: 7.25	PER 100: 7.25
BATCH: 500	TIME: 7.26	PER 100: 7.26
BEST PER 100: 7.25


### Factorized Basset (1000)

In [25]:
model = factorized_basset_model(seqlen=1000, num_outputs=1)

In [26]:
%%capture --no-stdout
time_ig(model, [100, 200, 500], 1000, num_examples=10, targets=[None])

BATCH: 100	TIME: 0.82	PER 100: 8.18
BATCH: 200	TIME: 0.80	PER 100: 8.00
BATCH: 500	TIME: 0.80	PER 100: 8.04
BEST PER 100: 8.00


In [27]:
%%capture --no-stdout
time_ig(model, [100, 200, 500], 1000, num_examples=100, targets=[None])

BATCH: 100	TIME: 8.13	PER 100: 8.13
BATCH: 200	TIME: 7.90	PER 100: 7.90
BATCH: 500	TIME: 7.97	PER 100: 7.97
BEST PER 100: 7.90


### Factorized Basset (2000)

In [28]:
model = factorized_basset_model(seqlen=2000, num_outputs=1)

In [29]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=10, targets=[None])

BATCH: 100	TIME: 1.60	PER 100: 16.02
BATCH: 200	TIME: 1.58	PER 100: 15.77
BATCH: 300	TIME: 1.61	PER 100: 16.11
BEST PER 100: 15.77


In [30]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=100, targets=[None])

BATCH: 100	TIME: 16.09	PER 100: 16.09
BATCH: 200	TIME: 16.01	PER 100: 16.01
BATCH: 300	TIME: 16.32	PER 100: 16.32
BEST PER 100: 16.01


In [31]:
%%capture --no-stdout
time_ig(model, [100, 200, 300], 2000, num_examples=200, targets=[None])

BATCH: 100	TIME: 32.41	PER 100: 16.20
BATCH: 200	TIME: 32.08	PER 100: 16.04
BATCH: 300	TIME: 32.82	PER 100: 16.41
BEST PER 100: 16.04


### BPNet (1000)

In [9]:
model = bpnet_model(seqlen=1000, num_dilated_convs=9)

# flatten and concat outputs
inp = tf.keras.Input(shape=model.input_shape[1:])
prof, cts = model(inp)
prof = tf.keras.layers.Flatten()(prof)
cts = tf.keras.layers.Flatten()(cts)
out = tf.keras.layers.Concatenate()([prof, cts])
model_ig = tf.keras.Model(inputs=inp, outputs=out)

# flattened outputs
model = model_ig

In [10]:
model.output

<tf.Tensor 'concatenate/concat:0' shape=(None, 1001) dtype=float32>

In [11]:
time_ig(model, [500], 1000, num_examples=10, targets=range(1001)) # all 1000 profile outs + 1 count out

BATCH: 500	TIME: 536.40	PER 100: 5364.00
BEST PER 100: 5364.00


### BPNet (2000)

In [13]:
model = bpnet_model(seqlen=2000, num_dilated_convs=9)

# flatten and concat outputs
inp = tf.keras.Input(shape=model.input_shape[1:])
prof, cts = model(inp)
prof = tf.keras.layers.Flatten()(prof)
cts = tf.keras.layers.Flatten()(cts)
out = tf.keras.layers.Concatenate()([prof, cts])
model_ig = tf.keras.Model(inputs=inp, outputs=out)

# flattened outputs
model = model_ig

In [14]:
model.output

<tf.Tensor 'concatenate_1/concat:0' shape=(None, 2001) dtype=float32>

In [16]:
time_ig(model, [500], 2000, num_examples=5, targets=range(2001)) # all 2000 profile outs + 1 count out

BATCH: 500	TIME: 874.62	PER 100: 17492.43
BEST PER 100: 17492.43
